In [1]:
import glob
import json
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import nibabel as nib
from nilearn.glm.first_level import FirstLevelModel
from nilearn.glm.first_level import make_first_level_design_matrix
from nilearn.plotting import plot_design_matrix, plot_stat_map
import os
import pandas as pd
import pickle
import re
import copy

import sys
sys.path.append('/Users/zeynepenkavi/Documents/RangelLab/DescribedVsLearned_fmri/analysis/01_level1')
from level1_utils import run_level1, make_level1_design_matrix, get_events

/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-dvl-fmri/lib/python3.8/site-packages/nilearn/glm/__init__.py:55: FutureWarning: The nilearn.glm module is experimental. It may change in any future release of Nilearn.
  warn('The nilearn.glm module is experimental. '
/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-dvl-fmri/lib/python3.8/site-packages/nilearn/datasets/__init__.py:86: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [2]:
data_path = '/Users/zeynepenkavi/Downloads/GTavares_2017_arbitration/bids_nifti_wface/'
behavior_path = '/Users/zeynepenkavi/Downloads/GTavares_2017_arbitration/behavioral_data/all_trials.csv'

subnum = '01'
runnum='4'

to_filter = ['trans', 'rot', 'drift', 'framewise', 'scrub', 'constant', 'dvars']

In [16]:
# Read in fmri events
fn = os.path.join(data_path, 'sub-%s/func/sub-%s_task-bundles_run-%s_events.tsv' %(subnum, subnum, runnum))
events = pd.read_csv(fn, sep='\t')

# Read in behavioral data with modeled value and RPE estimates
behavior = pd.read_csv(behavior_path)

# Extract the correct subnum and runnum from behavioral data
run_behavior = behavior.query('subnum == %d & session == %d'%(int(subnum), int(runnum)))

# Demean columns that might be used for parametric regressors
demean_cols = ['probFractalDraw', 'leftBundleVal', 'rightBundleVal', 'leftLotteryEV', 'rightLotteryEV', 'leftQValue', 'rightQValue', 'reward', 'ppe', 'leftQVAdv', 'leftEVAdv', 'leftbundleValAdv', 'leftFractalRpe', 'rightFractalRpe', 'valChosen', 'valUnchosen', 'valChosenLottery', 'valUnchosenLottery', 'valChosenFractal', 'valUnchosenFractal', 'leftFractalReward', 'rightFractalReward']
demean_df = run_behavior[demean_cols]
demean_df = demean_df - demean_df.mean()

In [17]:
demean_df[['valChosen', 'valUnchosen']]

,valChosen,valUnchosen
180,-0.035897,-0.059248
181,0.206427,0.141012
182,0.016918,0.048047
183,-0.008076,-0.068948
184,0.014258,-0.163475
185,-0.020241,0.141470
186,0.013192,-0.009358
187,0.380167,0.141506
188,0.356350,0.138291
189,0.052353,0.084426


In [4]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model1', data_path, behavior_path, regress_rt = 0)
# plot_design_matrix(design_matrix)
beh_regs = design_matrix.columns
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

A 'modulation' column was found in the given events data and is used.


/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-dvl-fmri/lib/python3.8/site-packages/nilearn/_utils/glm.py:310: UserWarning: Matrix is singular at working precision, regularizing...
  warn('Matrix is singular at working precision, regularizing...')


,choiceShift_st,fractalProb_ev,reward_ev,stim_ev
choiceShift_st,1.000000,0.590639,0.757393,0.837817
fractalProb_ev,0.590639,1.000000,0.027088,0.645227
reward_ev,0.757393,0.027088,1.000000,0.498400
stim_ev,0.837817,0.645227,0.498400,1.000000


In [5]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model1a', data_path, behavior_path, regress_rt = 0)
beh_regs = design_matrix.columns
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

A 'modulation' column was found in the given events data and is used.


/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-dvl-fmri/lib/python3.8/site-packages/nilearn/_utils/glm.py:310: UserWarning: Matrix is singular at working precision, regularizing...
  warn('Matrix is singular at working precision, regularizing...')


,choiceShiftLeft_st,choiceShiftRight_st,fractalProb_ev,reward_ev,stim_ev
choiceShiftLeft_st,1.000000,-0.351427,0.290387,0.389383,0.482793
choiceShiftRight_st,-0.351427,1.000000,0.379695,0.471358,0.473862
fractalProb_ev,0.290387,0.379695,1.000000,0.027088,0.645227
reward_ev,0.389383,0.471358,0.027088,1.000000,0.498400
stim_ev,0.482793,0.473862,0.645227,0.498400,1.000000


In [6]:
design_matrix = make_level1_design_matrix(subnum, runnum, 'model2', data_path, behavior_path, regress_rt = 0)
beh_regs = design_matrix.columns
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

A 'modulation' column was found in the given events data and is used.


/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-dvl-fmri/lib/python3.8/site-packages/nilearn/_utils/glm.py:310: UserWarning: Matrix is singular at working precision, regularizing...
  warn('Matrix is singular at working precision, regularizing...')


,choiceShift_st,fractalProb_ev,fractalProb_par,reward_ev,stim_ev
choiceShift_st,1.000000,0.590639,0.024667,0.757393,0.837817
fractalProb_ev,0.590639,1.000000,0.006043,0.027088,0.645227
fractalProb_par,0.024667,0.006043,1.000000,0.032470,-0.098054
reward_ev,0.757393,0.027088,0.032470,1.000000,0.498400
stim_ev,0.837817,0.645227,-0.098054,0.498400,1.000000


In [7]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model3', data_path, behavior_path, regress_rt = 0)
beh_regs = design_matrix.columns
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

A 'modulation' column was found in the given events data and is used.


/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-dvl-fmri/lib/python3.8/site-packages/nilearn/_utils/glm.py:310: UserWarning: Matrix is singular at working precision, regularizing...
  warn('Matrix is singular at working precision, regularizing...')


,choiceShift_st,fractalProb_ev,fractalProb_par,reward_ev,stim_ev,valDiff_par
choiceShift_st,1.000000,0.590639,0.024667,0.757393,0.837817,0.060696
fractalProb_ev,0.590639,1.000000,0.006043,0.027088,0.645227,0.043132
fractalProb_par,0.024667,0.006043,1.000000,0.032470,-0.098054,-0.159301
reward_ev,0.757393,0.027088,0.032470,1.000000,0.498400,0.029220
stim_ev,0.837817,0.645227,-0.098054,0.498400,1.000000,0.130875
valDiff_par,0.060696,0.043132,-0.159301,0.029220,0.130875,1.000000


In [8]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model4', data_path, behavior_path, regress_rt = 0)
beh_regs = design_matrix.columns
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

A 'modulation' column was found in the given events data and is used.


/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-dvl-fmri/lib/python3.8/site-packages/nilearn/_utils/glm.py:310: UserWarning: Matrix is singular at working precision, regularizing...
  warn('Matrix is singular at working precision, regularizing...')


,choiceShift_st,fractalProb_ev,fractalProb_par,reward_ev,stim_ev,valChosen_par,valUnchosen_par
choiceShift_st,1.000000,0.590639,0.024667,0.757393,0.837817,-0.044498,0.037792
fractalProb_ev,0.590639,1.000000,0.006043,0.027088,0.645227,-0.030753,0.022965
fractalProb_par,0.024667,0.006043,1.000000,0.032470,-0.098054,-0.301764,-0.126303
reward_ev,0.757393,0.027088,0.032470,1.000000,0.498400,-0.019469,0.013940
stim_ev,0.837817,0.645227,-0.098054,0.498400,1.000000,-0.176986,0.132436
valChosen_par,-0.044498,-0.030753,-0.301764,-0.019469,-0.176986,1.000000,0.242648
valUnchosen_par,0.037792,0.022965,-0.126303,0.013940,0.132436,0.242648,1.000000


In [9]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model4', data_path, behavior_path, regress_rt = 0, hrf_model=None, drift_model=None)
beh_regs = design_matrix.columns
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

A 'modulation' column was found in the given events data and is used.


/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-dvl-fmri/lib/python3.8/site-packages/nilearn/_utils/glm.py:310: UserWarning: Matrix is singular at working precision, regularizing...
  warn('Matrix is singular at working precision, regularizing...')


,choiceShift_st,fractalProb_ev,fractalProb_par,reward_ev,stim_ev,valChosen_par,valUnchosen_par
choiceShift_st,1.000000e+00,-1.876402e-02,-5.282643e-15,9.278736e-02,-1.208398e-02,5.825958e-04,-1.467576e-03
fractalProb_ev,-1.876402e-02,1.000000e+00,-1.507562e-13,-2.022260e-01,-1.020359e-01,4.919383e-03,-1.239207e-02
fractalProb_par,-5.282643e-15,-1.507562e-13,1.000000e+00,2.293617e-14,-8.030047e-14,-1.185609e-13,-1.039239e-13
reward_ev,9.278736e-02,-2.022260e-01,2.293617e-14,1.000000e+00,-1.302330e-01,6.278827e-03,-1.581655e-02
stim_ev,-1.208398e-02,-1.020359e-01,-8.030047e-14,-1.302330e-01,1.000000e+00,-4.821226e-02,1.214481e-01
valChosen_par,5.825958e-04,4.919383e-03,-1.185609e-13,6.278827e-03,-4.821226e-02,1.000000e+00,2.879616e-01
valUnchosen_par,-1.467576e-03,-1.239207e-02,-1.039239e-13,-1.581655e-02,1.214481e-01,2.879616e-01,1.000000e+00


In [10]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model5', data_path, behavior_path, regress_rt = 0)
beh_regs = design_matrix.columns
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

A 'modulation' column was found in the given events data and is used.


/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-dvl-fmri/lib/python3.8/site-packages/nilearn/_utils/glm.py:310: UserWarning: Matrix is singular at working precision, regularizing...
  warn('Matrix is singular at working precision, regularizing...')


,choiceShift_st,fractalProb_ev,fractalProb_par,reward_ev,stim_ev,valDiffFractal_par,valDiffLottery_par
choiceShift_st,1.000000,0.590639,0.024667,0.757393,0.837817,0.066875,0.001632
fractalProb_ev,0.590639,1.000000,0.006043,0.027088,0.645227,0.048696,-0.001519
fractalProb_par,0.024667,0.006043,1.000000,0.032470,-0.098054,-0.055300,0.047054
reward_ev,0.757393,0.027088,0.032470,1.000000,0.498400,0.034670,-0.001794
stim_ev,0.837817,0.645227,-0.098054,0.498400,1.000000,0.145922,0.009642
valDiffFractal_par,0.066875,0.048696,-0.055300,0.034670,0.145922,1.000000,-0.082635
valDiffLottery_par,0.001632,-0.001519,0.047054,-0.001794,0.009642,-0.082635,1.000000


In [11]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model5a', data_path, behavior_path, regress_rt = 0)
beh_regs = design_matrix.columns
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

A 'modulation' column was found in the given events data and is used.


/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-dvl-fmri/lib/python3.8/site-packages/nilearn/_utils/glm.py:310: UserWarning: Matrix is singular at working precision, regularizing...
  warn('Matrix is singular at working precision, regularizing...')


,choiceShift_st,fractalProb_ev,fractalProb_par,reward_ev,stim_ev,valDiffFractalWeighted_par,valDiffLotteryWeighted_par
choiceShift_st,1.000000,0.590639,0.024667,0.757393,0.837817,0.008549,-0.005146
fractalProb_ev,0.590639,1.000000,0.006043,0.027088,0.645227,0.001036,-0.003685
fractalProb_par,0.024667,0.006043,1.000000,0.032470,-0.098054,0.010299,0.085413
reward_ev,0.757393,0.027088,0.032470,1.000000,0.498400,-0.004034,-0.004963
stim_ev,0.837817,0.645227,-0.098054,0.498400,1.000000,0.139205,-0.005466
valDiffFractalWeighted_par,0.008549,0.001036,0.010299,-0.004034,0.139205,1.000000,0.007395
valDiffLotteryWeighted_par,-0.005146,-0.003685,0.085413,-0.004963,-0.005466,0.007395,1.000000


In [12]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model5b', data_path, behavior_path, regress_rt = 0)
beh_regs = design_matrix.columns
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

A 'modulation' column was found in the given events data and is used.


/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-dvl-fmri/lib/python3.8/site-packages/nilearn/_utils/glm.py:310: UserWarning: Matrix is singular at working precision, regularizing...
  warn('Matrix is singular at working precision, regularizing...')


,choiceShift_st,fractalProb_ev,fractalProb_par,reward_ev,stim_ev,valDiffFractalHighPFrac_par,valDiffFractalLowPFrac_par,valDiffFractalMedPFrac_par,valDiffLotteryHighPFrac_par,valDiffLotteryLowPFrac_par,valDiffLotteryMedPFrac_par
choiceShift_st,1.000000,0.590639,0.024667,0.757393,0.837817,0.042387,0.086212,-0.068118,-0.005873,-0.071553,0.052195
fractalProb_ev,0.590639,1.000000,0.006043,0.027088,0.645227,0.025391,0.069478,-0.050327,-0.009865,-0.054140,0.039609
fractalProb_par,0.024667,0.006043,1.000000,0.032470,-0.098054,0.033067,-0.114490,-0.024351,-0.033413,0.094793,0.029488
reward_ev,0.757393,0.027088,0.032470,1.000000,0.498400,0.013470,0.054741,-0.035756,-0.003997,-0.043650,0.027343
stim_ev,0.837817,0.645227,-0.098054,0.498400,1.000000,0.213766,0.048159,-0.125460,-0.015391,-0.076721,0.091500
valDiffFractalHighPFrac_par,0.042387,0.025391,0.033067,0.013470,0.213766,1.000000,-0.018410,-0.009124,-0.096257,0.019759,0.010689
valDiffFractalLowPFrac_par,0.086212,0.069478,-0.114490,0.054741,0.048159,-0.018410,1.000000,-0.050995,-0.001998,-0.220486,-0.021388
valDiffFractalMedPFrac_par,-0.068118,-0.050327,-0.024351,-0.035756,-0.125460,-0.009124,-0.050995,1.000000,-0.000174,-0.021989,-0.079617
valDiffLotteryHighPFrac_par,-0.005873,-0.009865,-0.033413,-0.003997,-0.015391,-0.096257,-0.001998,-0.000174,1.000000,0.002453,-0.020237
valDiffLotteryLowPFrac_par,-0.071553,-0.054140,0.094793,-0.043650,-0.076721,0.019759,-0.220486,-0.021989,0.002453,1.000000,0.005228


In [13]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model6', data_path, behavior_path, regress_rt = 0)
beh_regs = design_matrix.columns
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

A 'modulation' column was found in the given events data and is used.


/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-dvl-fmri/lib/python3.8/site-packages/nilearn/_utils/glm.py:310: UserWarning: Matrix is singular at working precision, regularizing...
  warn('Matrix is singular at working precision, regularizing...')


,choiceShift_st,fractalProb_ev,fractalProb_par,reward_ev,stim_ev,valChosenFractal_par,valChosenLottery_par,valUnchosenFractal_par,valUnchosenLottery_par
choiceShift_st,1.000000,0.590639,0.024667,0.757393,0.837817,-0.082667,0.004560,0.017815,-0.001653
fractalProb_ev,0.590639,1.000000,0.006043,0.027088,0.645227,-0.059632,0.003536,0.011191,-0.005233
fractalProb_par,0.024667,0.006043,1.000000,0.032470,-0.098054,0.132682,-0.103114,-0.257019,0.156511
reward_ev,0.757393,0.027088,0.032470,1.000000,0.498400,-0.041971,0.003610,0.010327,-0.005687
stim_ev,0.837817,0.645227,-0.098054,0.498400,1.000000,-0.179108,-0.027442,0.012674,0.037566
valChosenFractal_par,-0.082667,-0.059632,0.132682,-0.041971,-0.179108,1.000000,-0.179703,-0.358223,-0.068888
valChosenLottery_par,0.004560,0.003536,-0.103114,0.003610,-0.027442,-0.179703,1.000000,0.340287,0.146566
valUnchosenFractal_par,0.017815,0.011191,-0.257019,0.010327,0.012674,-0.358223,0.340287,1.000000,0.095349
valUnchosenLottery_par,-0.001653,-0.005233,0.156511,-0.005687,0.037566,-0.068888,0.146566,0.095349,1.000000


In [14]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model6a', data_path, behavior_path, regress_rt = 0)
beh_regs = design_matrix.columns
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

A 'modulation' column was found in the given events data and is used.


/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-dvl-fmri/lib/python3.8/site-packages/nilearn/_utils/glm.py:310: UserWarning: Matrix is singular at working precision, regularizing...
  warn('Matrix is singular at working precision, regularizing...')


,choiceShift_st,fractalProb_ev,fractalProb_par,reward_ev,stim_ev,valChosenFractalWeighted_par,valChosenLotteryWeighted_par,valUnchosenFractalWeighted_par,valUnchosenLotteryWeighted_par
choiceShift_st,1.000000,0.590639,0.024667,0.757393,0.837817,-0.031685,0.060930,-0.110515,-0.070206
fractalProb_ev,0.590639,1.000000,0.006043,0.027088,0.645227,-0.017365,0.050101,-0.090110,-0.056846
fractalProb_par,0.024667,0.006043,1.000000,0.032470,-0.098054,0.006434,-0.012493,-0.132760,0.150565
reward_ev,0.757393,0.027088,0.032470,1.000000,0.498400,-0.004524,0.039048,-0.069798,-0.047680
stim_ev,0.837817,0.645227,-0.098054,0.498400,1.000000,-0.220213,0.011181,-0.083087,-0.020182
valChosenFractalWeighted_par,-0.031685,-0.017365,0.006434,-0.004524,-0.220213,1.000000,-0.131186,-0.201514,-0.050459
valChosenLotteryWeighted_par,0.060930,0.050101,-0.012493,0.039048,0.011181,-0.131186,1.000000,0.146774,0.282308
valUnchosenFractalWeighted_par,-0.110515,-0.090110,-0.132760,-0.069798,-0.083087,-0.201514,0.146774,1.000000,-0.049618
valUnchosenLotteryWeighted_par,-0.070206,-0.056846,0.150565,-0.047680,-0.020182,-0.050459,0.282308,-0.049618,1.000000


In [15]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model7', data_path, behavior_path, regress_rt = 0)
beh_regs = design_matrix.columns
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

A 'modulation' column was found in the given events data and is used.


/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-dvl-fmri/lib/python3.8/site-packages/nilearn/_utils/glm.py:310: UserWarning: Matrix is singular at working precision, regularizing...
  warn('Matrix is singular at working precision, regularizing...')


,choiceShift_st,fractalProb_ev,fractalProb_par,ppe_par,rewardLeftFractal_par,rewardRightFractal_par,reward_ev,reward_par,rpeLeftFractal_par,rpeRightFractal_par,stim_ev,valDiffFractal_par,valDiffLottery_par
choiceShift_st,1.000000,0.590639,0.024667,0.000411,-0.006272,0.009882,0.757393,0.004689,0.000666,0.002853,0.837817,0.066875,0.001632
fractalProb_ev,0.590639,1.000000,0.006043,-0.008455,-0.014289,0.001477,0.027088,-0.009622,0.009383,0.003109,0.645227,0.048696,-0.001519
fractalProb_par,0.024667,0.006043,1.000000,-0.113123,-0.062747,-0.002642,0.032470,-0.178029,-0.025199,0.019728,-0.098054,-0.055300,0.047054
ppe_par,0.000411,-0.008455,-0.113123,1.000000,-0.136530,-0.192608,0.000125,-0.094356,-0.165469,0.066219,0.037734,0.039986,-0.048166
rewardLeftFractal_par,-0.006272,-0.014289,-0.062747,-0.136530,1.000000,-0.142021,-0.006842,0.312869,0.006223,0.029644,0.042765,-0.051056,0.039623
rewardRightFractal_par,0.009882,0.001477,-0.002642,-0.192608,-0.142021,1.000000,0.011357,0.344418,-0.096840,-0.014179,0.019232,0.091403,-0.038319
reward_ev,0.757393,0.027088,0.032470,0.000125,-0.006842,0.011357,1.000000,0.003282,-0.001887,0.003412,0.498400,0.034670,-0.001794
reward_par,0.004689,-0.009622,-0.178029,-0.094356,0.312869,0.344418,0.003282,1.000000,0.144203,0.021148,0.061959,0.195878,0.043703
rpeLeftFractal_par,0.000666,0.009383,-0.025199,-0.165469,0.006223,-0.096840,-0.001887,0.144203,1.000000,-0.152386,-0.022419,0.100152,-0.000409
rpeRightFractal_par,0.002853,0.003109,0.019728,0.066219,0.029644,-0.014179,0.003412,0.021148,-0.152386,1.000000,-0.000323,-0.166622,-0.099636


In [ ]:
mnum = 'model1'
reg_rt = 1
out_path = os.path.join(data_path, "derivatives/nilearn/glm/level1/%s_reg-rt%s")%(mnum, str(reg_rt)

# run_level1(subnum, mnum, data_path, behavior_path, out_path, regress_rt = reg_rt)

In [ ]:
mnum = 'model2'
reg_rt = 0
out_path = os.path.join(data_path, "derivatives/nilearn/glm/level1/%s_reg-rt%s")%(mnum, str(reg_rt))
                                                                                
# run_level1(subnum, 'model2', data_path, behavior_path, out_path, regress_rt = 0)